# Customer Service Tuning with FLOR-1.3B and QLoRA

In this Jupyter Notebook, we embark on an advanced project focused on fine-tuning customer service capabilities using a unique combination of the FLOR-1.3B model and QLoRA methodology. This approach is particularly notable for its application to the Catalan language, a less commonly supported language in the realm of large language models (LLMs). Given the challenge of finding models adept in minority languages like Catalan, the choice of FLOR-1.3B, a model fundamentally trained with Catalan data, is strategic and innovative.

The FLOR-1.3B model, a derivative of the larger BLOOM model family, is fine-tuned here using QLoRA, a technique designed for efficient and effective training of large models.

In this notebook, our primary goal is to enhance customer service interactions in Catalan, leveraging the specialized capabilities of the FLOR-1.3B model. This involves fine-tuning the model with a focus on understanding and generating responses in Catalan, addressing the needs of a specific user base more effectively. We utilize the QLoRA methodology to achieve this in a resource-efficient manner, ensuring that the model's performance is optimized for our specific use case.

Throughout the notebook, we cover various aspects of this process, including setting up the necessary dependencies, loading the model efficiently, and implementing training and evaluation strategies. We also focus on the nuances of working with a language like Catalan, understanding its unique linguistic features, and ensuring that the model's performance is tailored to the needs of the Catalan-speaking community.

This project not only demonstrates the technical prowess of combining FLOR-1.3B with QLoRA but also highlights the importance of language diversity in the field of AI and machine learning. By focusing on a minority language, we contribute to a more inclusive and accessible technological landscape.

## 1. Installing dependencies and log in to Hugging Face

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

This code imports the PyTorch library and checks if a CUDA-compatible GPU is available, which is crucial for accelerating machine learning tasks.

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

## 2. Loading the base model and the tokenizer

This code snippet is configuring a language model for processing tasks. It loads a specific model, projecte-aina/FLOR-1.3B-Instructed, optimized for efficient computation and hardware compatibility. Additionally, it sets up a tokenizer, essential for converting text into a format the model can process, with specific padding configurations.

In [ ]:
model_id = "projecte-aina/FLOR-1.3B-Instructed"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

In [ ]:
print(model)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(50257, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear4bit(in_features=2048, out_features=6144, bias=True)
          (dense): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear4bit(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear4bit(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affin


This is the description of the model architecture.

* Word Embeddings: Converts each word into a 2048-dimensional vector, providing numerical representations of words.

* Layer Normalization: Applied to embeddings and other layers to stabilize training.

* Bloom Blocks (0-23): The model consists of 24 layers (BloomBlock).

* Self-attention mechanism: Helps the model to focus on different parts of the input sequence for understanding context.

* Multi-Layer Perceptron (MLP): Consists of linear layers with a GELU activation function, aiding in complex pattern recognition.

* Linear Layers: The final linear layer maps the high-dimensional output back to the vocabulary size (50257), enabling the model to predict the next word in a sequence.

## 3. Loading the dataset

The dataset is a Catalan version of the **bitext/Bitext-customer-support-llm-chatbot-training-dataset**, which is likely a collection of data used to train customer support chatbots.

The translation from the original dataset to Catalan was performed using the **Helsinki-NLP's** Catalan translation model. Helsinki-NLP models are widely recognized for their effectiveness in machine translation tasks and are available on platforms like Hugging Face.

In [ ]:
from datasets import load_dataset

dataset_name = "ericrisco/customer_service_chatbot_ca"
dataset = load_dataset(dataset_name)

Generating train split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 450
    })
    test: Dataset({
        features: ['instruction', 'response'],
        num_rows: 50
    })
})


In [ ]:
dataset["test"] = dataset["train"].select(range(200))

## 4. Prompt preparation


These functions format data into structured prompts for language model training or evaluation. **prompt_template** constructs a text block with predefined instructions, user input as context, and a response if available. **create_prompt** processes a data sample, extracting instruction and response, and uses **prompt_template** to format them. This setup is useful for training conversational AI, ensuring consistent data presentation.

In [ ]:
system_message = "Below is an instruction that describes a task. You are a helpfull customer service assistant. Answer always in catalan."

def prompt_template(input, response):

  full_prompt = "<|startoftext|>"
  full_prompt += "\n### Instruction\n"
  full_prompt += system_message + "\n\n"
  full_prompt += "### Context\n"
  full_prompt += input + "\n\n"
  full_prompt += "### Answer\n"
  if response != "":
    full_prompt += response
    full_prompt += "\n<|endoftext|>"

  return full_prompt

def create_prompt(sample):
  input = sample["instruction"]
  response = sample["response"]

  full_prompt = prompt_template(input, response)

  return full_prompt

In [ ]:
prompt = create_prompt(dataset["train"][0])
print(prompt)

<s>
### Instruction
Below is an instruction that describes a task. You are a helpfull customer service assistant. Answer always in catalan.

### Context
Reemborsament dels diners

### Response
Entenc el seu desig de rebre el reemborsament dels seus diners. Obtenir un reemborsament pot ser un procés senzill, i estic aquí per guiar-lo a través d'ell. Per comenar, si us plau, recollir tota la informació necessària relacionada amb el seu pagament, com ara els detalls de la transacció, mètode de pagament utilitzat, i qualsevol documentació de suport rellevant. Una vegada que tingui aquesta informació.
</s>


## 5. Inference before training

The **generate_response** function uses a given language model and tokenizer to generate text responses from prompts. It encodes the prompt, processes it with the model, and then decodes the generated output. Key steps include encoding the prompt for the model, generating a response with a set maximum length and probabilistic sampling, and finally decoding and returning the response, minus the original prompt.

In [ ]:
def generate_response(prompt, model, tokenizer):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=256, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [ ]:
response = generate_response(prompt_template("Com puc accedir a la meva factura?", ""),
                  model,
                  tokenizer)
print(response)

Si un client o algun de vosaltres us demana que l'assessoreu, o bé en aquest cas, com que no vau veure la vostra factura, és molt probable que, a més del formulari “El meu compte” que hi ha quan t'envien un e-mail amb la factura, també hi hagi un altre formulari, en aquest cas “Què faig si la meva factura no és vàlida“ el qual us permetrà conèixer i corregir els possibles errors que hagin pogut aparèixer als e-mails que ens van enviar, i tot allò que calgui per a poder rebre una factura correcta. Per a poder accedir a la facturació, a la pàgina “El meu compte” haureu de prémer el botó de menú (els tres punts), i en la part de “Facturació” haureu de fer la cerca: Per a localitzar la factura, heu d'apujar el ratolí i arrossegar l'i per a buscar-la. I un cop la trobeu, us podreu descarregar la factura des d'ací mateix o des d'altres opcions que us surten a la pantalla, com pot ser “Registra’t en Factura, Factura, i Verifica la factura”. I en aquest cas, us caldrà la contrasenya d'accés.



## 6. Finetuning preparation

In [ ]:
from peft import prepare_model_for_kbit_training
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 12582912 || all params: 720136192 || trainable%: 1.747296155891579


In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(50257, 2048)
        (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=6144, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=6144, bias=False)
                )
               

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "flor-1-3B-customerservice",
  #num_train_epochs=1,
  max_steps = 500,
  per_device_train_batch_size = 4,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20,
  learning_rate=2e-4,
  #bf16=True,
  lr_scheduler_type='constant',
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,
  args=args,
  train_dataset=dataset["train"],
  eval_dataset=dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


## 7. Training and uploading to Hugging Face

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
20,0.841200,0.808956
40,0.747300,0.713816
60,0.673300,0.632729
80,0.596000,0.547042
100,0.508300,0.471045
120,0.441500,0.404085
140,0.376700,0.350677
160,0.324900,0.287352
180,0.273900,0.243814
200,0.231900,0.201622


Checkpoint destination directory flor-1-3B-customerservice/checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory flor-1-3B-customerservice/checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated 

Step,Training Loss,Validation Loss
20,0.841200,0.808956
40,0.747300,0.713816
60,0.673300,0.632729
80,0.596000,0.547042
100,0.508300,0.471045
120,0.441500,0.404085
140,0.376700,0.350677
160,0.324900,0.287352
180,0.273900,0.243814
200,0.231900,0.201622


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=500, training_loss=0.26305633491277697, metrics={'train_runtime': 2035.6882, 'train_samples_per_second': 0.982, 'train_steps_per_second': 0.246, 'total_flos': 2.836129879425024e+16, 'train_loss': 0.26305633491277697, 'epoch': 55.56})

In [ ]:
merged_model = model.merge_and_unload()
merged_model.push_to_hub("ericrisco/flor-1-3B-customerservice")
tokenizer.push_to_hub("ericrisco/flor-1-3B-customerservice")

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ericrisco/flor-1-3B-customerservice/commit/50e79f6d23578dd94b902628fdc0fc43884c05a3', commit_message='Upload tokenizer', commit_description='', oid='50e79f6d23578dd94b902628fdc0fc43884c05a3', pr_url=None, pr_revision=None, pr_num=None)